# Setup

In [1]:
# import libraries
import os
import sys
import time
import pandas as pd
import numpy as np
from scipy import stats
from scipy.interpolate import CubicSpline
import torch.optim as optim
import torch.nn as nn
import torch
from torch.optim import Adam
from scipy.stats import mode
from sklearn.preprocessing import LabelEncoder
import random
from sklearn.metrics import f1_score
from torch.utils.data import TensorDataset, DataLoader

## Hyperparameters

In [2]:
# set the seed
np.random.seed(420)
torch.manual_seed(420)
torch.cuda.manual_seed(420)
batch_size = 32

In [3]:
# set the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load Dataset

In [4]:
# load data without header
data = pd.read_csv('./ISWC21_data_plus_raw/rwhar_data.csv', header=None)
# add header
data.columns = ['subject_id', 'acc_x', 'acc_y', 'acc_z', 'activity']
data.head()

,subject_id,acc_x,acc_y,acc_z,activity
0,0,-9.57434,-2.02733,1.34506,climbing_up
1,0,-9.56479,-1.99597,1.39345,climbing_up
2,0,-9.55122,-1.98445,1.41139,climbing_up
3,0,-9.51335,-1.97557,1.42615,climbing_up
4,0,-9.52959,-1.98187,1.45395,climbing_up


In [5]:
# print the count of unique subjects
print('The number of unique subjects is {}'.format(data['subject_id'].nunique()))

The number of unique subjects is 15


In [6]:
data.shape

(3200803, 5)

In [7]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(data['activity'])
data['encoded_activity'] = encoded_labels

In [8]:
data.head()

,subject_id,acc_x,acc_y,acc_z,activity,encoded_activity
0,0,-9.57434,-2.02733,1.34506,climbing_up,1
1,0,-9.56479,-1.99597,1.39345,climbing_up,1
2,0,-9.55122,-1.98445,1.41139,climbing_up,1
3,0,-9.51335,-1.97557,1.42615,climbing_up,1
4,0,-9.52959,-1.98187,1.45395,climbing_up,1


In [9]:
# get number of classes
num_classes = data['encoded_activity'].nunique()
num_classes

8

In [10]:
# check the null values
data.isnull().sum()

subject_id          0
acc_x               0
acc_y               0
acc_z               0
activity            0
encoded_activity    0
dtype: int64

# Data Preprocessing

## Split Train and Test

In [11]:
# split train and test users
# randomly select 70% of the users for training
train_subjects = np.random.choice(data['subject_id'].unique(), int(0.7*len(data['subject_id'].unique())), replace=False)
# split the data into train and test
train = data[data['subject_id'].isin(train_subjects)]
test = data[~data['subject_id'].isin(train_subjects)]

# print test and train users
print('The number of train users is {}'.format(train['subject_id'].nunique()))
print('The number of test users is {}'.format(test['subject_id'].nunique()))

# print the shape of train and test
print('The shape of train is {}'.format(train.shape))
print('The shape of test is {}'.format(test.shape))

The number of train users is 10
The number of test users is 5
The shape of train is (2200794, 6)
The shape of test is (1000009, 6)


In [12]:
train_subjects

array([ 2, 14,  5,  0, 11,  4, 10, 12,  7, 13], dtype=int64)

In [13]:
# print the test subjects
print('The test subjects are {}'.format(test['subject_id'].unique()))

# [1 3 6 8 9] are the test subjects

The test subjects are [1 3 6 8 9]


## Data Normalization

In [14]:
# # setup the mean and std for normalization
# mean = {'acc_x': 0.816012, 'acc_y': -0.007595, 'acc_z': 0.074082}
# std = {'acc_x': 0.398664, 'acc_y': 0.375481, 'acc_z': 0.366527}

# # normalize the data for acc_x, acc_y, acc_z
# data['acc_x'] = (data['acc_x'] - mean['acc_x']) / std['acc_x']
# data['acc_y'] = (data['acc_y'] - mean['acc_y']) / std['acc_y']
# data['acc_z'] = (data['acc_z'] - mean['acc_z']) / std['acc_z']

In [15]:
# # z-normalize the data for acc_x, acc_y, acc_z with mean and std of train data
# train_data_mean = train[['acc_x', 'acc_y', 'acc_z']].mean()
# train_data_std = train[['acc_x', 'acc_y', 'acc_z']].std()

# # Normalize Training Data
# train.loc[:, ['acc_x', 'acc_y', 'acc_z']] = (train[['acc_x', 'acc_y', 'acc_z']] - train_data_mean) / train_data_std

# # Normalize Test Data with Training Statistics
# test.loc[:, ['acc_x', 'acc_y', 'acc_z']] = (test[['acc_x', 'acc_y', 'acc_z']] - train_data_mean) / train_data_std

## Generate Subset of Train Data

In [16]:
# randomly select 75% of the users for training
# train_subjects_75 = np.random.choice(train['subject_id'].unique(), int(0.75*len(train['subject_id'].unique())), replace=False)
train_subjects_75 = [2, 4, 5, 7, 12, 13, 14]
train_75 = data[data['subject_id'].isin(train_subjects_75)]

# print the train subjects
print('The train subjects are {}'.format(train_75['subject_id'].unique()))
#print shape of train_75
print('The shape of train_75 is {}'.format(train_75.shape))

The train subjects are [ 2  4  5  7 12 13 14]
The shape of train_75 is (1553420, 6)


In [17]:
# randomly select 50% of the users for training
# train_subjects_50 = np.random.choice(train['subject_id'].unique(), int(0.5*len(train['subject_id'].unique())), replace=False)
train_subjects_50 = [0, 2, 5, 12, 14]
train_50 = data[data['subject_id'].isin(train_subjects_50)]

# print the train subjects
print('The train subjects are {}'.format(train_50['subject_id'].unique()))
#print shape of train_50
print('The shape of train_50 is {}'.format(train_50.shape))

The train subjects are [ 0  2  5 12 14]
The shape of train_50 is (1099152, 6)


In [18]:
# randomly select 25% of the users for training
# train_subjects_25 = np.random.choice(train['subject_id'].unique(), int(0.25*len(train['subject_id'].unique())), replace=False)
train_subjects_25 = [5, 10]
train_25 = data[data['subject_id'].isin(train_subjects_25)]

# print the train subjects
print('The train subjects are {}'.format(train_25['subject_id'].unique()))
#print shape of train_25
print('The shape of train_25 is {}'.format(train_25.shape))

The train subjects are [ 5 10]
The shape of train_25 is (436182, 6)


In [19]:
# randomly select 10% of the users for training
# train_subjects_10 = np.random.choice(train['subject_id'].unique(), int(0.1*len(train['subject_id'].unique())), replace=False)
train_subjects_10 = [11]
train_10 = data[data['subject_id'].isin(train_subjects_10)]

# print the train subjects
print('The train subjects are {}'.format(train_10['subject_id'].unique()))
#print shape of train_10
print('The shape of train_10 is {}'.format(train_10.shape))

The train subjects are [11]
The shape of train_10 is (212453, 6)


## Windowing

In [20]:
def sliding_window_samples(data, samples_per_window, overlap_ratio):
    """
    Return a sliding window measured in number of samples over a data array along with the mode label for each window.

    :param data: input array, can be numpy or pandas dataframe
    :param samples_per_window: window length as number of samples
    :param overlap_ratio: overlap is meant as percentage and should be an integer value
    :return: tuple of windows, indices, and labels
    """
    windows = []
    indices = []
    labels = []
    curr = 0
    win_len = int(samples_per_window)
    if overlap_ratio is not None:
        overlapping_elements = int((overlap_ratio / 100) * win_len)
        if overlapping_elements >= win_len:
            print('Number of overlapping elements exceeds window size.')
            return
    while curr < len(data) - win_len:
        window = data[curr:curr + win_len]
        windows.append(window.iloc[:, :-2])  # Exclude the last two columns (original and encoded labels)
        indices.append([curr, curr + win_len])
        
        # Extract and compute the mode of the encoded labels for the current window
        window_labels = window['encoded_activity']
        mode_result = mode(window_labels)
        window_label = mode_result[0] if mode_result[0].size > 0 else mode_result
        labels.append(window_label)

        curr += win_len - overlapping_elements

    result_windows = np.array(windows)
    result_indices = np.array(indices)
    result_labels = np.array(labels)
    return result_windows, result_indices, result_labels

In [21]:
sampling_rate = 50
time_window = 8
window_size = sampling_rate * time_window
overlap_ratio = 50

train_window_data, _, train_window_label = sliding_window_samples(train, window_size, overlap_ratio)
print(f"shape of train window dataset ({time_window} sec with {overlap_ratio}% overlap): {train_window_data.shape}")

test_window_data, _, test_window_label = sliding_window_samples(test, window_size, overlap_ratio)
print(f"shape of test window dataset ({time_window} sec with {overlap_ratio}% overlap): {test_window_data.shape}")

shape of train window dataset (8 sec with 50% overlap): (11002, 400, 4)
shape of test window dataset (8 sec with 50% overlap): (4999, 400, 4)


In [22]:
train_window_data[0]

array([[ 0.     , -9.57434, -2.02733,  1.34506],
       [ 0.     , -9.56479, -1.99597,  1.39345],
       [ 0.     , -9.55122, -1.98445,  1.41139],
       ...,
       [ 0.     , -9.6026 , -1.59747,  1.52399],
       [ 0.     , -9.61617, -1.46892,  1.50549],
       [ 0.     , -9.59715, -1.51093,  1.40639]])

In [23]:
test_window_data[0]

array([[ 1.     , -9.35294, -1.67323,  2.46066],
       [ 1.     , -9.42616, -1.75624,  2.50548],
       [ 1.     , -9.47662, -1.78891,  2.5455 ],
       ...,
       [ 1.     , -9.11644, -1.72507,  3.24915],
       [ 1.     , -9.10716, -1.6602 ,  3.28903],
       [ 1.     , -9.08749, -1.52971,  3.3084 ]])

In [24]:
# remove the subject column
train_window_data = train_window_data[:, :, 1:]
test_window_data = test_window_data[:, :, 1:]

In [25]:
# print the shape of train and test
print('The shape of train is {}'.format(train_window_data.shape))
print('The shape of test is {}'.format(test_window_data.shape))

The shape of train is (11002, 400, 3)
The shape of test is (4999, 400, 3)


In [26]:
# length of train and test label
print('The length of train label is {}'.format(len(train_window_label)))
print('The length of test label is {}'.format(len(test_window_label)))

The length of train label is 11002
The length of test label is 4999


### 75% Data

In [27]:
sampling_rate = 50
time_window = 8
window_size = sampling_rate * time_window
overlap_ratio = 50

train_window_data_75, _, train_window_label_75 = sliding_window_samples(train_75, window_size, overlap_ratio)
print(f"shape of train window dataset ({time_window} sec with {overlap_ratio}% overlap): {train_window_data_75.shape}")

shape of train window dataset (8 sec with 50% overlap): (7766, 400, 4)


In [28]:
# remove the subject column
train_window_data_75 = train_window_data_75[:, :, 1:]

In [29]:
print('The shape of train_75 is {}'.format(train_window_data_75.shape))
print('The length of train_75 label is {}'.format(len(train_window_label_75)))

The shape of train_75 is (7766, 400, 3)
The length of train_75 label is 7766


### 50%

In [30]:
train_window_data_50, _, train_window_label_50 = sliding_window_samples(train_50, window_size, overlap_ratio)
print(f"shape of train window dataset ({time_window} sec with {overlap_ratio}% overlap): {train_window_data_50.shape}")

# remove the subject column
train_window_data_50 = train_window_data_50[:, :, 1:]

print('The shape of train_50 is {}'.format(train_window_data_50.shape))
print('The length of train_50 label is {}'.format(len(train_window_label_50)))

shape of train window dataset (8 sec with 50% overlap): (5494, 400, 4)
The shape of train_50 is (5494, 400, 3)
The length of train_50 label is 5494


### 25%

In [31]:
train_window_data_25, _, train_window_label_25 = sliding_window_samples(train_25, window_size, overlap_ratio)
# remove the subject column
train_window_data_25 = train_window_data_25[:, :, 1:]

print('The shape of train_25 is {}'.format(train_window_data_25.shape))
print('The length of train_25 label is {}'.format(len(train_window_label_25)))

The shape of train_25 is (2179, 400, 3)
The length of train_25 label is 2179


### 10%

In [32]:
train_window_data_10, _, train_window_label_10 = sliding_window_samples(train_10, window_size, overlap_ratio)
# remove the subject column
train_window_data_10 = train_window_data_10[:, :, 1:]

print('The shape of train_10 is {}'.format(train_window_data_10.shape))
print('The length of train_10 label is {}'.format(len(train_window_label_10)))

The shape of train_10 is (1061, 400, 3)
The length of train_10 label is 1061


## Generate Subset of Training Data

In [33]:
# # Calculate the sample size
# sample_size = int(0.75 * len(train_window_data))

# # Generate random indices
# indices = random.sample(range(len(train_window_data)), sample_size)

# # Sample the data and labels
# sampled_train_window_data_75 = [train_window_data[i] for i in indices]
# sampled_train_window_label_75 = [train_window_label[i] for i in indices]

# # print the shape of sampled train data and label
# print('The shape of sampled train label is {}'.format(np.array(sampled_train_window_label_75).shape))
# print('The shape of sampled train is {}'.format(np.array(sampled_train_window_data_75).shape))


In [34]:
# # Calculate the sample size
# sample_size = int(0.5 * len(train_window_data))

# # Generate random indices
# indices = random.sample(range(len(train_window_data)), sample_size)

# # Sample the data and labels
# sampled_train_window_data_50 = [train_window_data[i] for i in indices]
# sampled_train_window_label_50 = [train_window_label[i] for i in indices]

# # print the shape of sampled train data and label
# print('The shape of sampled train label is {}'.format(np.array(sampled_train_window_label_50).shape))
# print('The shape of sampled train is {}'.format(np.array(sampled_train_window_data_50).shape))

In [35]:
# # Calculate the sample size
# sample_size = int(0.25 * len(train_window_data))

# # Generate random indices
# indices = random.sample(range(len(train_window_data)), sample_size)

# # Sample the data and labels
# sampled_train_window_data_25 = [train_window_data[i] for i in indices]
# sampled_train_window_label_25 = [train_window_label[i] for i in indices]

# # print the shape of sampled train data and label
# print('The shape of sampled train label is {}'.format(np.array(sampled_train_window_label_25).shape))
# print('The shape of sampled train is {}'.format(np.array(sampled_train_window_data_25).shape))


In [36]:
# # Calculate the sample size
# sample_size = int(0.1 * len(train_window_data))

# # Generate random indices
# indices = random.sample(range(len(train_window_data)), sample_size)

# # Sample the data and labels
# sampled_train_window_data_10 = [train_window_data[i] for i in indices]
# sampled_train_window_label_10 = [train_window_label[i] for i in indices]

# # print the shape of sampled train data and label
# print('The shape of sampled train label is {}'.format(np.array(sampled_train_window_label_10).shape))
# print('The shape of sampled train is {}'.format(np.array(sampled_train_window_data_10).shape))

## Generate dataloader

In [37]:
# generate dataloader for train and test
def generate_dataloader(data, label, batch_size, is_shuffle=True):
    """
    Generate dataloader for train and test

    :param data: input data
    :param label: input label
    :param batch_size: batch size
    :return: train and test dataloader
    """
    # Check if data and label are lists, and convert them to NumPy arrays if they are
    if isinstance(data, list):
        data = np.array(data)
    if isinstance(label, list):
        label = np.array(label)
    
    # Convert data and label to tensor
    data_tensor = torch.from_numpy(data).float()  # Ensure data is converted to float for PyTorch
    label_tensor = torch.from_numpy(label).long()  # Labels typically converted to long for classification tasks
    
    # Generate dataloader
    dataset = TensorDataset(data_tensor, label_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=is_shuffle)
    
    return dataloader


In [38]:
# generate dataloader for train and test
train_dataloader = generate_dataloader(train_window_data, train_window_label, batch_size)   
test_dataloader = generate_dataloader(test_window_data, test_window_label, batch_size, is_shuffle=False)

# # generate dataloader for train sampled data and label
# train_dataloader_75 = generate_dataloader(sampled_train_window_data_75, sampled_train_window_label_75, batch_size)
# train_dataloader_50 = generate_dataloader(sampled_train_window_data_50, sampled_train_window_label_50, batch_size)
# train_dataloader_25 = generate_dataloader(sampled_train_window_data_25, sampled_train_window_label_25, batch_size)
# train_dataloader_10 = generate_dataloader(sampled_train_window_data_10, sampled_train_window_label_10, batch_size)

# generate dataloader for train and test in subject subsetting
train_dataloader_75 = generate_dataloader(train_window_data_75, train_window_label_75, batch_size)
train_dataloader_50 = generate_dataloader(train_window_data_50, train_window_label_50, batch_size)
train_dataloader_25 = generate_dataloader(train_window_data_25, train_window_label_25, batch_size)
train_dataloader_10 = generate_dataloader(train_window_data_10, train_window_label_10, batch_size)

## Train and Test Functions

In [39]:
# create training function
def train_function(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs = inputs.transpose(1, 2)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
    return running_loss / len(train_loader)

In [40]:
def test_function(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for i, data in enumerate(test_loader):
            inputs, labels = data[0].to(device), data[1].to(device)
            inputs = inputs.transpose(1, 2)  # Assuming this is necessary for your model
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()

            # Collect all true labels and predictions for F1 score calculation
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    # Calculate accuracy
    accuracy = correct / len(test_loader.dataset)
    
    # Calculate F1 score. You might need to adjust the 'average' parameter based on your task
    # For binary classification, you can use 'binary'. For multi-class, consider 'macro' or 'weighted'
    f1 = f1_score(all_labels, all_predictions, average='weighted')

    return running_loss / len(test_loader), accuracy, f1


In [41]:
# create function to train and test model
def train_and_test(model, train_loader, test_loader, criterion, optimizer, device, num_epochs):
    train_losses = []
    test_losses = []
    test_accuracies = []
    test_f1_scores = []  # List to store F1-scores for each epoch

    for epoch in range(num_epochs):
        # Training phase
        train_loss = train_function(model, train_loader, criterion, optimizer, device)
        train_losses.append(train_loss)

        # Testing phase
        test_loss, test_accuracy, test_f1 = test_function(model, test_loader, criterion, device)  # Modified to receive F1-score
        test_losses.append(test_loss)
        test_accuracies.append(test_accuracy)
        test_f1_scores.append(test_f1)  # Store the F1-score

        # Print epoch summary
        print(f"Epoch: {epoch + 1}/{num_epochs}.. Train Loss: {train_loss:.3f}.. "
              f"Test Loss: {test_loss:.3f}.. Test Accuracy: {test_accuracy:.3f}.. Test F1 Score: {test_f1:.3f}")

    return train_losses, test_losses, test_accuracies, test_f1_scores

# Multitask Self Supervised

## Model Architecture

In [42]:
class TPN(nn.Module):
    def __init__(self):
        super(TPN, self).__init__()
        self.trunk = nn.Sequential(
            nn.Conv1d(in_channels=3, out_channels=32, kernel_size=24, stride=1),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=16, stride=1),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Conv1d(in_channels=64, out_channels=96, kernel_size=8, stride=1),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.AdaptiveMaxPool1d(output_size=1)
        )

        self.heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(96, 256),
                nn.ReLU(),
                nn.Linear(256, 1),
                nn.Sigmoid()
            ) for _ in range(8)  # 8 heads for 8 different transformations
        ])

    def forward(self, x):
        x = self.trunk(x)
        x = x.view(x.size(0), -1)  # Flatten the output for the fully-connected layer
        outputs = [head(x) for head in self.heads]
        return outputs

In [43]:
class SupervisedTPN(nn.Module):
    def __init__(self, pretrained_model, num_classes):
        super(SupervisedTPN, self).__init__()
        self.trunk = pretrained_model.trunk # Use the trunk from the pretrained model
        # Freeze the trunk
        for param in self.trunk.parameters():
            param.requires_grad = False

        self.head = nn.Sequential(
            nn.Linear(96, 1024),  # Adjusted to match the document's description
            nn.ReLU(),
            nn.Linear(1024, num_classes)  # Softmax applied externally during training
        )
        # No softmax here as it's included in nn.CrossEntropyLoss during training

    def forward(self, x):
        x = self.trunk(x)
        x = x.view(x.size(0), -1)  # Flatten the output for the fully-connected layer
        output = self.head(x)
        return output

## Fine-Tuning Only Head Layers

### Full Train Data

In [44]:
# load pre-trained model
pretrained_model_path = './multitask/tpn_30_epoch_regularized.pt'  # Adjust as necessary
pretrained_model = TPN()
pretrained_model.load_state_dict(torch.load(pretrained_model_path))

# Create the supervised model by adjusting the pre-trained model
supervised_model = SupervisedTPN(pretrained_model, num_classes=num_classes).to(device)

In [45]:
# Prepare for fine-tuning
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(supervised_model.head.parameters(), lr=0.0003, weight_decay=0.0001)

# train and test model
num_epochs = 30
train_losses, test_losses, test_accuracies, test_f1_scores = train_and_test(supervised_model, train_dataloader, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 0.948.. Test Loss: 1.105.. Test Accuracy: 0.640.. Test F1 Score: 0.626
Epoch: 2/30.. Train Loss: 0.704.. Test Loss: 1.074.. Test Accuracy: 0.682.. Test F1 Score: 0.679
Epoch: 3/30.. Train Loss: 0.674.. Test Loss: 1.119.. Test Accuracy: 0.648.. Test F1 Score: 0.644
Epoch: 4/30.. Train Loss: 0.631.. Test Loss: 1.136.. Test Accuracy: 0.662.. Test F1 Score: 0.656
Epoch: 5/30.. Train Loss: 0.617.. Test Loss: 1.104.. Test Accuracy: 0.674.. Test F1 Score: 0.678
Epoch: 6/30.. Train Loss: 0.586.. Test Loss: 1.008.. Test Accuracy: 0.676.. Test F1 Score: 0.671
Epoch: 7/30.. Train Loss: 0.565.. Test Loss: 1.033.. Test Accuracy: 0.674.. Test F1 Score: 0.675
Epoch: 8/30.. Train Loss: 0.548.. Test Loss: 1.135.. Test Accuracy: 0.689.. Test F1 Score: 0.676
Epoch: 9/30.. Train Loss: 0.541.. Test Loss: 1.034.. Test Accuracy: 0.681.. Test F1 Score: 0.678
Epoch: 10/30.. Train Loss: 0.529.. Test Loss: 1.098.. Test Accuracy: 0.675.. Test F1 Score: 0.674
Epoch: 11/30.. Train Loss: 0.

### 75% Train Data

In [46]:
# load pre-trained model
pretrained_model_path = './multitask/tpn_30_epoch_regularized.pt'  # Adjust as necessary
pretrained_model = TPN()
pretrained_model.load_state_dict(torch.load(pretrained_model_path))

# Create the supervised model by adjusting the pre-trained model
supervised_model = SupervisedTPN(pretrained_model, num_classes=num_classes).to(device)

In [47]:
# prepare for fine-tuning
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(supervised_model.head.parameters(), lr=0.0003, weight_decay=0.0001)

# train and test model
num_epochs = 30
train_losses, test_losses, test_accuracies, test_f1_scores = train_and_test(supervised_model, train_dataloader_75, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 1.021.. Test Loss: 1.168.. Test Accuracy: 0.641.. Test F1 Score: 0.636
Epoch: 2/30.. Train Loss: 0.763.. Test Loss: 1.137.. Test Accuracy: 0.632.. Test F1 Score: 0.633
Epoch: 3/30.. Train Loss: 0.684.. Test Loss: 1.119.. Test Accuracy: 0.642.. Test F1 Score: 0.641
Epoch: 4/30.. Train Loss: 0.656.. Test Loss: 1.371.. Test Accuracy: 0.653.. Test F1 Score: 0.646
Epoch: 5/30.. Train Loss: 0.625.. Test Loss: 1.398.. Test Accuracy: 0.657.. Test F1 Score: 0.649
Epoch: 6/30.. Train Loss: 0.607.. Test Loss: 1.226.. Test Accuracy: 0.655.. Test F1 Score: 0.654
Epoch: 7/30.. Train Loss: 0.605.. Test Loss: 1.351.. Test Accuracy: 0.651.. Test F1 Score: 0.649
Epoch: 8/30.. Train Loss: 0.559.. Test Loss: 1.258.. Test Accuracy: 0.656.. Test F1 Score: 0.653
Epoch: 9/30.. Train Loss: 0.547.. Test Loss: 1.254.. Test Accuracy: 0.652.. Test F1 Score: 0.664
Epoch: 10/30.. Train Loss: 0.532.. Test Loss: 1.394.. Test Accuracy: 0.645.. Test F1 Score: 0.643
Epoch: 11/30.. Train Loss: 0.

### 50% Train Data

In [48]:
# load pre-trained model
pretrained_model_path = './multitask/tpn_30_epoch_regularized.pt'  # Adjust as necessary
pretrained_model = TPN()
pretrained_model.load_state_dict(torch.load(pretrained_model_path))

# Create the supervised model by adjusting the pre-trained model
supervised_model = SupervisedTPN(pretrained_model, num_classes=num_classes).to(device)

In [49]:
# prepare for fine-tuning
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(supervised_model.head.parameters(), lr=0.0003, weight_decay=0.0001)

# train and test model
num_epochs = 30
train_losses, test_losses, test_accuracies, test_f1_scores = train_and_test(supervised_model, train_dataloader_50, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 1.074.. Test Loss: 1.251.. Test Accuracy: 0.558.. Test F1 Score: 0.565
Epoch: 2/30.. Train Loss: 0.721.. Test Loss: 1.103.. Test Accuracy: 0.645.. Test F1 Score: 0.640
Epoch: 3/30.. Train Loss: 0.650.. Test Loss: 1.163.. Test Accuracy: 0.620.. Test F1 Score: 0.620
Epoch: 4/30.. Train Loss: 0.584.. Test Loss: 1.003.. Test Accuracy: 0.688.. Test F1 Score: 0.681
Epoch: 5/30.. Train Loss: 0.585.. Test Loss: 1.089.. Test Accuracy: 0.671.. Test F1 Score: 0.671
Epoch: 6/30.. Train Loss: 0.534.. Test Loss: 1.069.. Test Accuracy: 0.664.. Test F1 Score: 0.661
Epoch: 7/30.. Train Loss: 0.545.. Test Loss: 1.024.. Test Accuracy: 0.673.. Test F1 Score: 0.678
Epoch: 8/30.. Train Loss: 0.513.. Test Loss: 1.154.. Test Accuracy: 0.663.. Test F1 Score: 0.663
Epoch: 9/30.. Train Loss: 0.498.. Test Loss: 1.008.. Test Accuracy: 0.697.. Test F1 Score: 0.703
Epoch: 10/30.. Train Loss: 0.488.. Test Loss: 1.200.. Test Accuracy: 0.619.. Test F1 Score: 0.623
Epoch: 11/30.. Train Loss: 0.

### 25% Train Data

In [50]:
# load pre-trained model
pretrained_model_path = './multitask/tpn_30_epoch_regularized.pt'  # Adjust as necessary
pretrained_model = TPN()
pretrained_model.load_state_dict(torch.load(pretrained_model_path))

# Create the supervised model by adjusting the pre-trained model
supervised_model = SupervisedTPN(pretrained_model, num_classes=num_classes).to(device)

In [51]:
# prepare for fine-tuning
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(supervised_model.head.parameters(), lr=0.0003, weight_decay=0.0001)

# train and test model
num_epochs = 30
train_losses, test_losses, test_accuracies, test_f1_scores = train_and_test(supervised_model, train_dataloader_25, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 0.943.. Test Loss: 1.586.. Test Accuracy: 0.599.. Test F1 Score: 0.596
Epoch: 2/30.. Train Loss: 0.699.. Test Loss: 1.695.. Test Accuracy: 0.582.. Test F1 Score: 0.580
Epoch: 3/30.. Train Loss: 0.441.. Test Loss: 1.578.. Test Accuracy: 0.582.. Test F1 Score: 0.585
Epoch: 4/30.. Train Loss: 0.428.. Test Loss: 1.546.. Test Accuracy: 0.572.. Test F1 Score: 0.573
Epoch: 5/30.. Train Loss: 0.397.. Test Loss: 1.618.. Test Accuracy: 0.575.. Test F1 Score: 0.573
Epoch: 6/30.. Train Loss: 0.390.. Test Loss: 1.587.. Test Accuracy: 0.574.. Test F1 Score: 0.575
Epoch: 7/30.. Train Loss: 0.393.. Test Loss: 1.666.. Test Accuracy: 0.529.. Test F1 Score: 0.527
Epoch: 8/30.. Train Loss: 0.342.. Test Loss: 1.912.. Test Accuracy: 0.578.. Test F1 Score: 0.580
Epoch: 9/30.. Train Loss: 0.334.. Test Loss: 2.034.. Test Accuracy: 0.534.. Test F1 Score: 0.527
Epoch: 10/30.. Train Loss: 0.327.. Test Loss: 1.769.. Test Accuracy: 0.583.. Test F1 Score: 0.582
Epoch: 11/30.. Train Loss: 0.

### 10% Train Data

In [52]:
# load pre-trained model
pretrained_model_path = './multitask/tpn_30_epoch_regularized.pt'  # Adjust as necessary
pretrained_model = TPN()
pretrained_model.load_state_dict(torch.load(pretrained_model_path))

# Create the supervised model by adjusting the pre-trained model
supervised_model = SupervisedTPN(pretrained_model, num_classes=num_classes).to(device)

In [53]:
# prepare for fine-tuning
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(supervised_model.head.parameters(), lr=0.0003, weight_decay=0.0001)

# train and test model
num_epochs = 30
train_losses, test_losses, test_accuracies, test_f1_scores = train_and_test(supervised_model, train_dataloader_10, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 1.208.. Test Loss: 1.652.. Test Accuracy: 0.541.. Test F1 Score: 0.533
Epoch: 2/30.. Train Loss: 0.560.. Test Loss: 1.709.. Test Accuracy: 0.476.. Test F1 Score: 0.470
Epoch: 3/30.. Train Loss: 0.462.. Test Loss: 1.730.. Test Accuracy: 0.508.. Test F1 Score: 0.505
Epoch: 4/30.. Train Loss: 0.525.. Test Loss: 1.722.. Test Accuracy: 0.473.. Test F1 Score: 0.479
Epoch: 5/30.. Train Loss: 0.344.. Test Loss: 1.909.. Test Accuracy: 0.453.. Test F1 Score: 0.454
Epoch: 6/30.. Train Loss: 0.297.. Test Loss: 1.846.. Test Accuracy: 0.474.. Test F1 Score: 0.483
Epoch: 7/30.. Train Loss: 0.269.. Test Loss: 1.988.. Test Accuracy: 0.467.. Test F1 Score: 0.472
Epoch: 8/30.. Train Loss: 0.265.. Test Loss: 1.946.. Test Accuracy: 0.478.. Test F1 Score: 0.485
Epoch: 9/30.. Train Loss: 0.241.. Test Loss: 2.100.. Test Accuracy: 0.469.. Test F1 Score: 0.471
Epoch: 10/30.. Train Loss: 0.216.. Test Loss: 2.014.. Test Accuracy: 0.478.. Test F1 Score: 0.481
Epoch: 11/30.. Train Loss: 0.

## Fine Tune Last Conv Layers

In [54]:
class SupervisedTPN(nn.Module):
    def __init__(self, pretrained_model, num_classes):
        super(SupervisedTPN, self).__init__()
        self.trunk = pretrained_model.trunk  # Use the trunk from the pretrained model

        # Freeze all the trunk layers first
        for param in self.trunk.parameters():
            param.requires_grad = False

        # Unfreeze the last convolutional layer
        # Assuming the last conv layer is the third from the last in the trunk sequence
        for param in self.trunk[-3].parameters():
            param.requires_grad = True

        self.head = nn.Sequential(
            nn.Linear(96, 1024),  # Adjusted to match the document's description
            nn.ReLU(),
            nn.Linear(1024, num_classes)  # Softmax applied externally during training
        )
        # No softmax here as it's included in nn.CrossEntropyLoss during training

    def forward(self, x):
        x = self.trunk(x)
        x = x.view(x.size(0), -1)  # Flatten the output for the fully-connected layer
        output = self.head(x)
        return output

### Full Train Data

In [55]:
# load pre-trained model
pretrained_model_path = './multitask/tpn_30_epoch_regularized.pt'  # Adjust as necessary
pretrained_model = TPN()
pretrained_model.load_state_dict(torch.load(pretrained_model_path))

# Create the supervised model by adjusting the pre-trained model
supervised_model = SupervisedTPN(pretrained_model, num_classes=num_classes).to(device)

In [56]:
# prepare for fine-tuning
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(supervised_model.parameters(), lr=0.0003, weight_decay=0.0001)

# train and test model
num_epochs = 30
train_losses, test_losses, test_accuracies, test_f1_scores = train_and_test(supervised_model, train_dataloader, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 0.946.. Test Loss: 1.021.. Test Accuracy: 0.650.. Test F1 Score: 0.645
Epoch: 2/30.. Train Loss: 0.717.. Test Loss: 1.114.. Test Accuracy: 0.651.. Test F1 Score: 0.645
Epoch: 3/30.. Train Loss: 0.662.. Test Loss: 1.099.. Test Accuracy: 0.661.. Test F1 Score: 0.659
Epoch: 4/30.. Train Loss: 0.635.. Test Loss: 1.371.. Test Accuracy: 0.654.. Test F1 Score: 0.653
Epoch: 5/30.. Train Loss: 0.608.. Test Loss: 1.049.. Test Accuracy: 0.680.. Test F1 Score: 0.674
Epoch: 6/30.. Train Loss: 0.587.. Test Loss: 1.084.. Test Accuracy: 0.669.. Test F1 Score: 0.664
Epoch: 7/30.. Train Loss: 0.570.. Test Loss: 0.949.. Test Accuracy: 0.687.. Test F1 Score: 0.683
Epoch: 8/30.. Train Loss: 0.553.. Test Loss: 1.011.. Test Accuracy: 0.676.. Test F1 Score: 0.675
Epoch: 9/30.. Train Loss: 0.538.. Test Loss: 1.152.. Test Accuracy: 0.683.. Test F1 Score: 0.680
Epoch: 10/30.. Train Loss: 0.532.. Test Loss: 1.102.. Test Accuracy: 0.661.. Test F1 Score: 0.665
Epoch: 11/30.. Train Loss: 0.

### 75% Train Data

In [57]:
# load pre-trained model
pretrained_model_path = './multitask/tpn_30_epoch_regularized.pt'  # Adjust as necessary
pretrained_model = TPN()
pretrained_model.load_state_dict(torch.load(pretrained_model_path))

# Create the supervised model by adjusting the pre-trained model
supervised_model = SupervisedTPN(pretrained_model, num_classes=num_classes).to(device)

In [58]:
# prepare for fine-tuning
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(supervised_model.parameters(), lr=0.0003, weight_decay=0.0001)

# train and test model
num_epochs = 30
train_losses_75, test_losses_75, test_accuracies_75, test_f1_scores_75 = train_and_test(supervised_model, train_dataloader_75, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 0.980.. Test Loss: 1.241.. Test Accuracy: 0.611.. Test F1 Score: 0.603
Epoch: 2/30.. Train Loss: 0.751.. Test Loss: 1.220.. Test Accuracy: 0.617.. Test F1 Score: 0.611
Epoch: 3/30.. Train Loss: 0.673.. Test Loss: 1.249.. Test Accuracy: 0.650.. Test F1 Score: 0.630
Epoch: 4/30.. Train Loss: 0.644.. Test Loss: 1.264.. Test Accuracy: 0.645.. Test F1 Score: 0.639
Epoch: 5/30.. Train Loss: 0.626.. Test Loss: 1.191.. Test Accuracy: 0.644.. Test F1 Score: 0.642
Epoch: 6/30.. Train Loss: 0.590.. Test Loss: 1.417.. Test Accuracy: 0.664.. Test F1 Score: 0.652
Epoch: 7/30.. Train Loss: 0.598.. Test Loss: 1.365.. Test Accuracy: 0.655.. Test F1 Score: 0.654
Epoch: 8/30.. Train Loss: 0.566.. Test Loss: 1.178.. Test Accuracy: 0.667.. Test F1 Score: 0.660
Epoch: 9/30.. Train Loss: 0.562.. Test Loss: 1.332.. Test Accuracy: 0.660.. Test F1 Score: 0.656
Epoch: 10/30.. Train Loss: 0.530.. Test Loss: 1.363.. Test Accuracy: 0.658.. Test F1 Score: 0.649
Epoch: 11/30.. Train Loss: 0.

### 50% Train Data

In [59]:
# load pre-trained model
pretrained_model_path = './multitask/tpn_30_epoch_regularized.pt'  # Adjust as necessary
pretrained_model = TPN()
pretrained_model.load_state_dict(torch.load(pretrained_model_path))

# Create the supervised model by adjusting the pre-trained model
supervised_model = SupervisedTPN(pretrained_model, num_classes=num_classes).to(device)

In [60]:
# prepare for fine-tuning
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(supervised_model.parameters(), lr=0.0003, weight_decay=0.0001)

# train and test model
num_epochs = 30
train_losses_50, test_losses_50, test_accuracies_50, test_f1_scores_50 = train_and_test(supervised_model, train_dataloader_50, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 0.986.. Test Loss: 1.137.. Test Accuracy: 0.638.. Test F1 Score: 0.628
Epoch: 2/30.. Train Loss: 0.702.. Test Loss: 1.073.. Test Accuracy: 0.680.. Test F1 Score: 0.662
Epoch: 3/30.. Train Loss: 0.634.. Test Loss: 1.181.. Test Accuracy: 0.660.. Test F1 Score: 0.657
Epoch: 4/30.. Train Loss: 0.601.. Test Loss: 1.050.. Test Accuracy: 0.688.. Test F1 Score: 0.686
Epoch: 5/30.. Train Loss: 0.568.. Test Loss: 1.086.. Test Accuracy: 0.686.. Test F1 Score: 0.683
Epoch: 6/30.. Train Loss: 0.571.. Test Loss: 0.946.. Test Accuracy: 0.709.. Test F1 Score: 0.709
Epoch: 7/30.. Train Loss: 0.535.. Test Loss: 1.005.. Test Accuracy: 0.707.. Test F1 Score: 0.695
Epoch: 8/30.. Train Loss: 0.539.. Test Loss: 1.083.. Test Accuracy: 0.690.. Test F1 Score: 0.695
Epoch: 9/30.. Train Loss: 0.511.. Test Loss: 1.027.. Test Accuracy: 0.707.. Test F1 Score: 0.712
Epoch: 10/30.. Train Loss: 0.500.. Test Loss: 0.975.. Test Accuracy: 0.681.. Test F1 Score: 0.670
Epoch: 11/30.. Train Loss: 0.

### 25% Train Data

In [61]:
# load pre-trained model
pretrained_model_path = './multitask/tpn_30_epoch_regularized.pt'  # Adjust as necessary
pretrained_model = TPN()
pretrained_model.load_state_dict(torch.load(pretrained_model_path))

# Create the supervised model by adjusting the pre-trained model
supervised_model = SupervisedTPN(pretrained_model, num_classes=num_classes).to(device)

In [62]:
# prepare for fine-tuning
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(supervised_model.parameters(), lr=0.0003, weight_decay=0.0001)

# train and test model
num_epochs = 30
train_losses_25, test_losses_25, test_accuracies_25, test_f1_scores_25 = train_and_test(supervised_model, train_dataloader_25, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 1.106.. Test Loss: 1.582.. Test Accuracy: 0.581.. Test F1 Score: 0.575
Epoch: 2/30.. Train Loss: 0.536.. Test Loss: 1.611.. Test Accuracy: 0.548.. Test F1 Score: 0.553
Epoch: 3/30.. Train Loss: 0.482.. Test Loss: 1.589.. Test Accuracy: 0.557.. Test F1 Score: 0.545
Epoch: 4/30.. Train Loss: 0.437.. Test Loss: 1.788.. Test Accuracy: 0.527.. Test F1 Score: 0.521
Epoch: 5/30.. Train Loss: 0.441.. Test Loss: 1.725.. Test Accuracy: 0.530.. Test F1 Score: 0.518
Epoch: 6/30.. Train Loss: 0.439.. Test Loss: 1.667.. Test Accuracy: 0.600.. Test F1 Score: 0.597
Epoch: 7/30.. Train Loss: 0.560.. Test Loss: 1.817.. Test Accuracy: 0.537.. Test F1 Score: 0.525
Epoch: 8/30.. Train Loss: 0.385.. Test Loss: 1.551.. Test Accuracy: 0.618.. Test F1 Score: 0.613
Epoch: 9/30.. Train Loss: 0.359.. Test Loss: 1.770.. Test Accuracy: 0.544.. Test F1 Score: 0.537
Epoch: 10/30.. Train Loss: 0.317.. Test Loss: 1.634.. Test Accuracy: 0.542.. Test F1 Score: 0.532
Epoch: 11/30.. Train Loss: 0.

### 10% Train Data

In [63]:
# load pre-trained model
pretrained_model_path = './multitask/tpn_30_epoch_regularized.pt'  # Adjust as necessary
pretrained_model = TPN()
pretrained_model.load_state_dict(torch.load(pretrained_model_path))

# Create the supervised model by adjusting the pre-trained model
supervised_model = SupervisedTPN(pretrained_model, num_classes=num_classes).to(device)

In [64]:
# prepare for fine-tuning
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(supervised_model.parameters(), lr=0.0003, weight_decay=0.0001)

# train and test model
num_epochs = 30
train_losses_10, test_losses_10, test_accuracies_10, test_f1_scores_10 = train_and_test(supervised_model, train_dataloader_10, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 1.476.. Test Loss: 2.021.. Test Accuracy: 0.387.. Test F1 Score: 0.359
Epoch: 2/30.. Train Loss: 0.606.. Test Loss: 1.853.. Test Accuracy: 0.444.. Test F1 Score: 0.450
Epoch: 3/30.. Train Loss: 0.488.. Test Loss: 1.752.. Test Accuracy: 0.441.. Test F1 Score: 0.440
Epoch: 4/30.. Train Loss: 0.378.. Test Loss: 1.926.. Test Accuracy: 0.451.. Test F1 Score: 0.446
Epoch: 5/30.. Train Loss: 0.340.. Test Loss: 1.953.. Test Accuracy: 0.452.. Test F1 Score: 0.447
Epoch: 6/30.. Train Loss: 0.269.. Test Loss: 1.924.. Test Accuracy: 0.449.. Test F1 Score: 0.450
Epoch: 7/30.. Train Loss: 0.271.. Test Loss: 2.094.. Test Accuracy: 0.462.. Test F1 Score: 0.465
Epoch: 8/30.. Train Loss: 0.259.. Test Loss: 1.923.. Test Accuracy: 0.470.. Test F1 Score: 0.461
Epoch: 9/30.. Train Loss: 0.261.. Test Loss: 2.364.. Test Accuracy: 0.454.. Test F1 Score: 0.446
Epoch: 10/30.. Train Loss: 0.252.. Test Loss: 2.114.. Test Accuracy: 0.459.. Test F1 Score: 0.445
Epoch: 11/30.. Train Loss: 0.

# RefuseAct

## Adjusting Window Size

In [65]:
# windowing data
sampling_rate = 50
time_window = 2
window_size = sampling_rate * time_window
overlap_ratio = 0

train_window_data, _, train_window_label = sliding_window_samples(train, window_size, overlap_ratio)
print(f"shape of train window dataset ({time_window} sec with {overlap_ratio}% overlap): {train_window_data.shape}")

test_window_data, _, test_window_label = sliding_window_samples(test, window_size, overlap_ratio)
print(f"shape of test window dataset ({time_window} sec with {overlap_ratio}% overlap): {test_window_data.shape}")

shape of train window dataset (2 sec with 0% overlap): (22007, 100, 4)
shape of test window dataset (2 sec with 0% overlap): (10000, 100, 4)


### Full Data

In [66]:
# remove the subject column
train_window_data = train_window_data[:, :, 1:]
test_window_data = test_window_data[:, :, 1:]

# print the shape of train and test
print('The shape of train is {}'.format(train_window_data.shape))
print('The shape of test is {}'.format(test_window_data.shape))

The shape of train is (22007, 100, 3)
The shape of test is (10000, 100, 3)


### 75% of data


In [67]:
train_window_data_75, _, train_window_label_75 = sliding_window_samples(train_75, window_size, overlap_ratio)
print(f"shape of train window dataset ({time_window} sec with {overlap_ratio}% overlap): {train_window_data_75.shape}")

# remove the subject column
train_window_data_75 = train_window_data_75[:, :, 1:]

print('The shape of train_75 is {}'.format(train_window_data_75.shape))
print('The length of train_75 label is {}'.format(len(train_window_label_75)))

shape of train window dataset (2 sec with 0% overlap): (15534, 100, 4)
The shape of train_75 is (15534, 100, 3)
The length of train_75 label is 15534


### 50% Data

In [68]:
train_window_data_50, _, train_window_label_50 = sliding_window_samples(train_50, window_size, overlap_ratio)
print(f"shape of train window dataset ({time_window} sec with {overlap_ratio}% overlap): {train_window_data_50.shape}")

# remove the subject column
train_window_data_50 = train_window_data_50[:, :, 1:]

print('The shape of train_50 is {}'.format(train_window_data_50.shape))
print('The length of train_50 label is {}'.format(len(train_window_label_50)))

shape of train window dataset (2 sec with 0% overlap): (10991, 100, 4)
The shape of train_50 is (10991, 100, 3)
The length of train_50 label is 10991


### 25% Data

In [69]:
train_window_data_25, _, train_window_label_25 = sliding_window_samples(train_25, window_size, overlap_ratio)
# remove the subject column
train_window_data_25 = train_window_data_25[:, :, 1:]

print('The shape of train_25 is {}'.format(train_window_data_25.shape))
print('The length of train_25 label is {}'.format(len(train_window_label_25)))

The shape of train_25 is (4361, 100, 3)
The length of train_25 label is 4361


### 10% Data

In [70]:
train_window_data_10, _, train_window_label_10 = sliding_window_samples(train_10, window_size, overlap_ratio)
# remove the subject column
train_window_data_10 = train_window_data_10[:, :, 1:]

print('The shape of train_10 is {}'.format(train_window_data_10.shape))
print('The length of train_10 label is {}'.format(len(train_window_label_10)))

The shape of train_10 is (2124, 100, 3)
The length of train_10 label is 2124


## Generate DataLoader

In [71]:
# generate dataloader for train and test
train_dataloader = generate_dataloader(train_window_data, train_window_label, batch_size)   
test_dataloader = generate_dataloader(test_window_data, test_window_label, batch_size, is_shuffle=False)

# generate dataloader for train and test in subject subsetting
train_dataloader_75 = generate_dataloader(train_window_data_75, train_window_label_75, batch_size)
train_dataloader_50 = generate_dataloader(train_window_data_50, train_window_label_50, batch_size)
train_dataloader_25 = generate_dataloader(train_window_data_25, train_window_label_25, batch_size)
train_dataloader_10 = generate_dataloader(train_window_data_10, train_window_label_10, batch_size)

## Network Architecture

### CNN

In [72]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNFeatureExtractor(nn.Module):
    def __init__(self, num_classes=4):
        super(CNNFeatureExtractor, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(256 * 12, 128)  # Adjust the input features according to your final conv layer output
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

### TCN

In [73]:
class TemporalBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, dilation, padding):
        super(TemporalBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size,
                               stride=stride, padding=0, dilation=dilation)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size,
                               stride=stride, padding=0, dilation=dilation)
        self.relu2 = nn.ReLU()
        self.downsample = nn.Conv1d(in_channels, out_channels, 1) if in_channels != out_channels else None
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.conv2(out)
        out = self.relu2(out)
        
        res = x if self.downsample is None else self.downsample(x)

        # Adjusting the length of the residual to match the output
        if out.size(2) != res.size(2):
            desired_length = out.size(2)
            res = res[:, :, :desired_length]

        return self.relu(out + res)


class TCN(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size, dropout=0.2, num_classes=4):
        super(TCN, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size + (dilation_size - 1))]

        self.tcn = nn.Sequential(*layers)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(num_channels[-1], num_classes)

    def forward(self, x):
        x = self.tcn(x)
        x = F.avg_pool1d(x, x.size(2)).squeeze(2)  # Global Average Pooling
        x = self.dropout(x)
        return self.fc(x)

## Fine Tune CNN

### Full Data

In [74]:
model = CNNFeatureExtractor(num_classes=4)

#load pretrained model
model.load_state_dict(torch.load('./models/cnn_feature_extractor_join_20231218-2006.pt'))

model.fc2 = nn.Linear(in_features=model.fc2.in_features, out_features=num_classes)
model.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc2.parameters(), lr=0.001)

# train and test model
num_epochs = 30
train_losses, test_losses, test_accuracies, test_f1_scores = train_and_test(model, train_dataloader, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 1.577.. Test Loss: 1.294.. Test Accuracy: 0.574.. Test F1 Score: 0.545
Epoch: 2/30.. Train Loss: 1.224.. Test Loss: 1.173.. Test Accuracy: 0.617.. Test F1 Score: 0.581
Epoch: 3/30.. Train Loss: 1.123.. Test Loss: 1.130.. Test Accuracy: 0.635.. Test F1 Score: 0.622
Epoch: 4/30.. Train Loss: 1.070.. Test Loss: 1.123.. Test Accuracy: 0.642.. Test F1 Score: 0.639
Epoch: 5/30.. Train Loss: 1.038.. Test Loss: 1.090.. Test Accuracy: 0.649.. Test F1 Score: 0.632
Epoch: 6/30.. Train Loss: 1.014.. Test Loss: 1.050.. Test Accuracy: 0.653.. Test F1 Score: 0.628
Epoch: 7/30.. Train Loss: 0.993.. Test Loss: 1.064.. Test Accuracy: 0.654.. Test F1 Score: 0.641
Epoch: 8/30.. Train Loss: 0.980.. Test Loss: 1.072.. Test Accuracy: 0.675.. Test F1 Score: 0.663
Epoch: 9/30.. Train Loss: 0.968.. Test Loss: 1.044.. Test Accuracy: 0.669.. Test F1 Score: 0.665
Epoch: 10/30.. Train Loss: 0.959.. Test Loss: 1.059.. Test Accuracy: 0.627.. Test F1 Score: 0.627
Epoch: 11/30.. Train Loss: 0.

### 75% Train Data

In [75]:
model = CNNFeatureExtractor(num_classes=4)

#load pretrained model
model.load_state_dict(torch.load('./models/cnn_feature_extractor_join_20231218-2006.pt'))

model.fc2 = nn.Linear(in_features=model.fc2.in_features, out_features=num_classes)
model.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc2.parameters(), lr=0.001)

# train and test model
num_epochs = 30
train_losses, test_losses, test_accuracies, test_f1_scores = train_and_test(model, train_dataloader_75, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 1.716.. Test Loss: 1.438.. Test Accuracy: 0.546.. Test F1 Score: 0.487
Epoch: 2/30.. Train Loss: 1.321.. Test Loss: 1.209.. Test Accuracy: 0.638.. Test F1 Score: 0.602
Epoch: 3/30.. Train Loss: 1.200.. Test Loss: 1.179.. Test Accuracy: 0.636.. Test F1 Score: 0.620
Epoch: 4/30.. Train Loss: 1.140.. Test Loss: 1.185.. Test Accuracy: 0.576.. Test F1 Score: 0.556
Epoch: 5/30.. Train Loss: 1.104.. Test Loss: 1.117.. Test Accuracy: 0.652.. Test F1 Score: 0.619
Epoch: 6/30.. Train Loss: 1.076.. Test Loss: 1.088.. Test Accuracy: 0.639.. Test F1 Score: 0.599
Epoch: 7/30.. Train Loss: 1.056.. Test Loss: 1.111.. Test Accuracy: 0.633.. Test F1 Score: 0.631
Epoch: 8/30.. Train Loss: 1.038.. Test Loss: 1.095.. Test Accuracy: 0.670.. Test F1 Score: 0.669
Epoch: 9/30.. Train Loss: 1.024.. Test Loss: 1.069.. Test Accuracy: 0.666.. Test F1 Score: 0.651
Epoch: 10/30.. Train Loss: 1.012.. Test Loss: 1.098.. Test Accuracy: 0.657.. Test F1 Score: 0.642
Epoch: 11/30.. Train Loss: 1.

### 50% Train Data

In [76]:
model = CNNFeatureExtractor(num_classes=4)

#load pretrained model
model.load_state_dict(torch.load('./models/cnn_feature_extractor_join_20231218-2006.pt'))

model.fc2 = nn.Linear(in_features=model.fc2.in_features, out_features=num_classes)
model.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc2.parameters(), lr=0.001)

# train and test model
num_epochs = 30
train_losses, test_losses, test_accuracies, test_f1_scores = train_and_test(model, train_dataloader_50, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 1.738.. Test Loss: 1.493.. Test Accuracy: 0.575.. Test F1 Score: 0.530
Epoch: 2/30.. Train Loss: 1.378.. Test Loss: 1.307.. Test Accuracy: 0.591.. Test F1 Score: 0.574
Epoch: 3/30.. Train Loss: 1.237.. Test Loss: 1.233.. Test Accuracy: 0.544.. Test F1 Score: 0.507
Epoch: 4/30.. Train Loss: 1.165.. Test Loss: 1.166.. Test Accuracy: 0.608.. Test F1 Score: 0.564
Epoch: 5/30.. Train Loss: 1.114.. Test Loss: 1.145.. Test Accuracy: 0.595.. Test F1 Score: 0.579
Epoch: 6/30.. Train Loss: 1.079.. Test Loss: 1.116.. Test Accuracy: 0.641.. Test F1 Score: 0.627
Epoch: 7/30.. Train Loss: 1.055.. Test Loss: 1.085.. Test Accuracy: 0.636.. Test F1 Score: 0.606
Epoch: 8/30.. Train Loss: 1.028.. Test Loss: 1.097.. Test Accuracy: 0.621.. Test F1 Score: 0.615
Epoch: 9/30.. Train Loss: 1.014.. Test Loss: 1.081.. Test Accuracy: 0.639.. Test F1 Score: 0.639
Epoch: 10/30.. Train Loss: 0.995.. Test Loss: 1.096.. Test Accuracy: 0.573.. Test F1 Score: 0.563
Epoch: 11/30.. Train Loss: 0.

### 25% Train Data

In [77]:
model = CNNFeatureExtractor(num_classes=4)

#load pretrained model
model.load_state_dict(torch.load('./models/cnn_feature_extractor_join_20231218-2006.pt'))

model.fc2 = nn.Linear(in_features=model.fc2.in_features, out_features=num_classes)
model.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc2.parameters(), lr=0.001)

# train and test model
num_epochs = 30
train_losses, test_losses, test_accuracies, test_f1_scores = train_and_test(model, train_dataloader_25, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 1.979.. Test Loss: 1.756.. Test Accuracy: 0.417.. Test F1 Score: 0.338
Epoch: 2/30.. Train Loss: 1.597.. Test Loss: 1.577.. Test Accuracy: 0.432.. Test F1 Score: 0.376
Epoch: 3/30.. Train Loss: 1.402.. Test Loss: 1.494.. Test Accuracy: 0.448.. Test F1 Score: 0.368
Epoch: 4/30.. Train Loss: 1.281.. Test Loss: 1.407.. Test Accuracy: 0.502.. Test F1 Score: 0.450
Epoch: 5/30.. Train Loss: 1.192.. Test Loss: 1.343.. Test Accuracy: 0.534.. Test F1 Score: 0.521
Epoch: 6/30.. Train Loss: 1.121.. Test Loss: 1.290.. Test Accuracy: 0.558.. Test F1 Score: 0.544
Epoch: 7/30.. Train Loss: 1.066.. Test Loss: 1.266.. Test Accuracy: 0.551.. Test F1 Score: 0.535
Epoch: 8/30.. Train Loss: 1.017.. Test Loss: 1.255.. Test Accuracy: 0.546.. Test F1 Score: 0.537
Epoch: 9/30.. Train Loss: 0.986.. Test Loss: 1.222.. Test Accuracy: 0.572.. Test F1 Score: 0.530
Epoch: 10/30.. Train Loss: 0.955.. Test Loss: 1.208.. Test Accuracy: 0.583.. Test F1 Score: 0.558
Epoch: 11/30.. Train Loss: 0.

### 10% Train Data

In [78]:
model = CNNFeatureExtractor(num_classes=4)

#load pretrained model
model.load_state_dict(torch.load('./models/cnn_feature_extractor_join_20231218-2006.pt'))

model.fc2 = nn.Linear(in_features=model.fc2.in_features, out_features=num_classes)
model.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc2.parameters(), lr=0.001)

# train and test model
num_epochs = 30
train_losses, test_losses, test_accuracies, test_f1_scores = train_and_test(model, train_dataloader_10, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 2.118.. Test Loss: 1.909.. Test Accuracy: 0.321.. Test F1 Score: 0.268
Epoch: 2/30.. Train Loss: 1.779.. Test Loss: 1.777.. Test Accuracy: 0.393.. Test F1 Score: 0.352
Epoch: 3/30.. Train Loss: 1.611.. Test Loss: 1.673.. Test Accuracy: 0.505.. Test F1 Score: 0.478
Epoch: 4/30.. Train Loss: 1.470.. Test Loss: 1.564.. Test Accuracy: 0.597.. Test F1 Score: 0.571
Epoch: 5/30.. Train Loss: 1.357.. Test Loss: 1.529.. Test Accuracy: 0.551.. Test F1 Score: 0.525
Epoch: 6/30.. Train Loss: 1.272.. Test Loss: 1.490.. Test Accuracy: 0.533.. Test F1 Score: 0.503
Epoch: 7/30.. Train Loss: 1.197.. Test Loss: 1.422.. Test Accuracy: 0.552.. Test F1 Score: 0.537
Epoch: 8/30.. Train Loss: 1.132.. Test Loss: 1.395.. Test Accuracy: 0.576.. Test F1 Score: 0.567
Epoch: 9/30.. Train Loss: 1.077.. Test Loss: 1.376.. Test Accuracy: 0.597.. Test F1 Score: 0.584
Epoch: 10/30.. Train Loss: 1.032.. Test Loss: 1.376.. Test Accuracy: 0.578.. Test F1 Score: 0.565
Epoch: 11/30.. Train Loss: 1.

## Fine Tune CNN Conv-3

### Full Train Data

In [79]:
model = CNNFeatureExtractor(num_classes=4)

#load pretrained model
model.load_state_dict(torch.load('./models/cnn_feature_extractor_join_20231218-2006.pt'))

# Freezing layers up to conv3
for name, param in model.named_parameters():
    if 'conv3' in name:
        break
    param.requires_grad = False

# Unfreeze layers from conv3 onwards
unfreeze = False
for name, param in model.named_parameters():
    if 'conv3' in name:
        unfreeze = True
    if unfreeze:
        param.requires_grad = True

model.fc2 = nn.Linear(in_features=model.fc2.in_features, out_features=num_classes)
model.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc2.parameters(), lr=0.001)

# train and test model
num_epochs = 30
train_losses, test_losses, test_accuracies, test_f1_scores = train_and_test(model, train_dataloader, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 1.574.. Test Loss: 1.317.. Test Accuracy: 0.533.. Test F1 Score: 0.492
Epoch: 2/30.. Train Loss: 1.221.. Test Loss: 1.167.. Test Accuracy: 0.589.. Test F1 Score: 0.564
Epoch: 3/30.. Train Loss: 1.119.. Test Loss: 1.125.. Test Accuracy: 0.638.. Test F1 Score: 0.625
Epoch: 4/30.. Train Loss: 1.068.. Test Loss: 1.092.. Test Accuracy: 0.614.. Test F1 Score: 0.597
Epoch: 5/30.. Train Loss: 1.038.. Test Loss: 1.081.. Test Accuracy: 0.657.. Test F1 Score: 0.655
Epoch: 6/30.. Train Loss: 1.010.. Test Loss: 1.081.. Test Accuracy: 0.616.. Test F1 Score: 0.611
Epoch: 7/30.. Train Loss: 0.991.. Test Loss: 1.061.. Test Accuracy: 0.628.. Test F1 Score: 0.612
Epoch: 8/30.. Train Loss: 0.978.. Test Loss: 1.043.. Test Accuracy: 0.671.. Test F1 Score: 0.667
Epoch: 9/30.. Train Loss: 0.964.. Test Loss: 1.031.. Test Accuracy: 0.655.. Test F1 Score: 0.650
Epoch: 10/30.. Train Loss: 0.955.. Test Loss: 1.026.. Test Accuracy: 0.685.. Test F1 Score: 0.676
Epoch: 11/30.. Train Loss: 0.

### 75% Train Data

In [80]:
model = CNNFeatureExtractor(num_classes=4)

#load pretrained model
model.load_state_dict(torch.load('./models/cnn_feature_extractor_join_20231218-2006.pt'))

# Freezing layers up to conv3
for name, param in model.named_parameters():
    if 'conv3' in name:
        break
    param.requires_grad = False

# Unfreeze layers from conv3 onwards
unfreeze = False
for name, param in model.named_parameters():
    if 'conv3' in name:
        unfreeze = True
    if unfreeze:
        param.requires_grad = True

model.fc2 = nn.Linear(in_features=model.fc2.in_features, out_features=num_classes)
model.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc2.parameters(), lr=0.001)

# train and test model
num_epochs = 30
train_losses_75, test_losses_75, test_accuracies_75, test_f1_scores_75 = train_and_test(model, train_dataloader_75, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 1.653.. Test Loss: 1.385.. Test Accuracy: 0.553.. Test F1 Score: 0.506
Epoch: 2/30.. Train Loss: 1.304.. Test Loss: 1.224.. Test Accuracy: 0.631.. Test F1 Score: 0.595
Epoch: 3/30.. Train Loss: 1.193.. Test Loss: 1.180.. Test Accuracy: 0.632.. Test F1 Score: 0.608
Epoch: 4/30.. Train Loss: 1.134.. Test Loss: 1.133.. Test Accuracy: 0.639.. Test F1 Score: 0.627
Epoch: 5/30.. Train Loss: 1.098.. Test Loss: 1.123.. Test Accuracy: 0.618.. Test F1 Score: 0.615
Epoch: 6/30.. Train Loss: 1.072.. Test Loss: 1.112.. Test Accuracy: 0.647.. Test F1 Score: 0.621
Epoch: 7/30.. Train Loss: 1.051.. Test Loss: 1.095.. Test Accuracy: 0.662.. Test F1 Score: 0.632
Epoch: 8/30.. Train Loss: 1.034.. Test Loss: 1.082.. Test Accuracy: 0.674.. Test F1 Score: 0.655
Epoch: 9/30.. Train Loss: 1.021.. Test Loss: 1.078.. Test Accuracy: 0.631.. Test F1 Score: 0.631
Epoch: 10/30.. Train Loss: 1.008.. Test Loss: 1.065.. Test Accuracy: 0.645.. Test F1 Score: 0.618
Epoch: 11/30.. Train Loss: 0.

### 50% Train Data

In [81]:
model = CNNFeatureExtractor(num_classes=4)

#load pretrained model
model.load_state_dict(torch.load('./models/cnn_feature_extractor_join_20231218-2006.pt'))

# Freezing layers up to conv3
for name, param in model.named_parameters():
    if 'conv3' in name:
        break
    param.requires_grad = False

# Unfreeze layers from conv3 onwards
unfreeze = False
for name, param in model.named_parameters():
    if 'conv3' in name:
        unfreeze = True
    if unfreeze:
        param.requires_grad = True

model.fc2 = nn.Linear(in_features=model.fc2.in_features, out_features=num_classes)
model.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# train and test model
num_epochs = 30
train_losses_50, test_losses_50, test_accuracies_50, test_f1_scores_50 = train_and_test(model, train_dataloader_50, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 0.989.. Test Loss: 1.019.. Test Accuracy: 0.660.. Test F1 Score: 0.658
Epoch: 2/30.. Train Loss: 0.621.. Test Loss: 1.132.. Test Accuracy: 0.641.. Test F1 Score: 0.651
Epoch: 3/30.. Train Loss: 0.552.. Test Loss: 1.165.. Test Accuracy: 0.596.. Test F1 Score: 0.601
Epoch: 4/30.. Train Loss: 0.483.. Test Loss: 1.100.. Test Accuracy: 0.665.. Test F1 Score: 0.665
Epoch: 5/30.. Train Loss: 0.444.. Test Loss: 1.162.. Test Accuracy: 0.633.. Test F1 Score: 0.631
Epoch: 6/30.. Train Loss: 0.426.. Test Loss: 1.176.. Test Accuracy: 0.655.. Test F1 Score: 0.653
Epoch: 7/30.. Train Loss: 0.394.. Test Loss: 1.181.. Test Accuracy: 0.659.. Test F1 Score: 0.665
Epoch: 8/30.. Train Loss: 0.376.. Test Loss: 1.230.. Test Accuracy: 0.657.. Test F1 Score: 0.663
Epoch: 9/30.. Train Loss: 0.361.. Test Loss: 1.363.. Test Accuracy: 0.658.. Test F1 Score: 0.663
Epoch: 10/30.. Train Loss: 0.334.. Test Loss: 1.290.. Test Accuracy: 0.666.. Test F1 Score: 0.672
Epoch: 11/30.. Train Loss: 0.

### 25% Train Data

In [82]:
model = CNNFeatureExtractor(num_classes=4)

#load pretrained model
model.load_state_dict(torch.load('./models/cnn_feature_extractor_join_20231218-2006.pt'))

# Freezing layers up to conv3
for name, param in model.named_parameters():
    if 'conv3' in name:
        break
    param.requires_grad = False

# Unfreeze layers from conv3 onwards
unfreeze = False
for name, param in model.named_parameters():
    if 'conv3' in name:
        unfreeze = True
    if unfreeze:
        param.requires_grad = True

model.fc2 = nn.Linear(in_features=model.fc2.in_features, out_features=num_classes)
model.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# train and test model
num_epochs = 30
train_losses_25, test_losses_25, test_accuracies_25, test_f1_scores_25 = train_and_test(model, train_dataloader_25, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 0.987.. Test Loss: 1.550.. Test Accuracy: 0.575.. Test F1 Score: 0.570
Epoch: 2/30.. Train Loss: 0.501.. Test Loss: 1.775.. Test Accuracy: 0.512.. Test F1 Score: 0.538
Epoch: 3/30.. Train Loss: 0.401.. Test Loss: 1.817.. Test Accuracy: 0.589.. Test F1 Score: 0.612
Epoch: 4/30.. Train Loss: 0.347.. Test Loss: 2.180.. Test Accuracy: 0.504.. Test F1 Score: 0.528
Epoch: 5/30.. Train Loss: 0.332.. Test Loss: 1.862.. Test Accuracy: 0.562.. Test F1 Score: 0.584
Epoch: 6/30.. Train Loss: 0.292.. Test Loss: 2.052.. Test Accuracy: 0.521.. Test F1 Score: 0.525
Epoch: 7/30.. Train Loss: 0.284.. Test Loss: 2.180.. Test Accuracy: 0.529.. Test F1 Score: 0.523
Epoch: 8/30.. Train Loss: 0.265.. Test Loss: 2.140.. Test Accuracy: 0.505.. Test F1 Score: 0.510
Epoch: 9/30.. Train Loss: 0.216.. Test Loss: 2.477.. Test Accuracy: 0.535.. Test F1 Score: 0.549
Epoch: 10/30.. Train Loss: 0.247.. Test Loss: 2.248.. Test Accuracy: 0.573.. Test F1 Score: 0.589
Epoch: 11/30.. Train Loss: 0.

### 10% Train Data

In [83]:
model = CNNFeatureExtractor(num_classes=4)

#load pretrained model
model.load_state_dict(torch.load('./models/cnn_feature_extractor_join_20231218-2006.pt'))

# Freezing layers up to conv3
for name, param in model.named_parameters():
    if 'conv3' in name:
        break
    param.requires_grad = False

# Unfreeze layers from conv3 onwards
unfreeze = False
for name, param in model.named_parameters():
    if 'conv3' in name:
        unfreeze = True
    if unfreeze:
        param.requires_grad = True

model.fc2 = nn.Linear(in_features=model.fc2.in_features, out_features=num_classes)
model.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# train and test model
num_epochs = 30
train_losses_10, test_losses_10, test_accuracies_10, test_f1_scores_10 = train_and_test(model, train_dataloader_10, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 1.283.. Test Loss: 2.248.. Test Accuracy: 0.410.. Test F1 Score: 0.464
Epoch: 2/30.. Train Loss: 0.432.. Test Loss: 2.290.. Test Accuracy: 0.450.. Test F1 Score: 0.478
Epoch: 3/30.. Train Loss: 0.315.. Test Loss: 3.315.. Test Accuracy: 0.316.. Test F1 Score: 0.345
Epoch: 4/30.. Train Loss: 0.250.. Test Loss: 2.807.. Test Accuracy: 0.357.. Test F1 Score: 0.358
Epoch: 5/30.. Train Loss: 0.201.. Test Loss: 3.039.. Test Accuracy: 0.401.. Test F1 Score: 0.443
Epoch: 6/30.. Train Loss: 0.169.. Test Loss: 3.396.. Test Accuracy: 0.375.. Test F1 Score: 0.387
Epoch: 7/30.. Train Loss: 0.162.. Test Loss: 2.783.. Test Accuracy: 0.417.. Test F1 Score: 0.429
Epoch: 8/30.. Train Loss: 0.130.. Test Loss: 3.568.. Test Accuracy: 0.421.. Test F1 Score: 0.411
Epoch: 9/30.. Train Loss: 0.130.. Test Loss: 3.146.. Test Accuracy: 0.439.. Test F1 Score: 0.433
Epoch: 10/30.. Train Loss: 0.127.. Test Loss: 3.088.. Test Accuracy: 0.389.. Test F1 Score: 0.397
Epoch: 11/30.. Train Loss: 0.

## Fine Tune TCN

### Full Train Data

In [84]:
num_inputs = 3  # Assuming 3 input channels (x, y, z axes of the accelerometer)
num_channels = [64, 128, 256]  # Example channel sizes for each layer
kernel_size = 8  # Kernel size for temporal convolutions

model = TCN(num_inputs, num_channels, kernel_size, num_classes=4)

#load pretrained model
model.load_state_dict(torch.load('./models/tcn_join_20231218-2014.pt'))

model.fc = nn.Linear(in_features=model.fc.in_features, out_features=num_classes)
model.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)

# train and test model
num_epochs = 30
train_losses, test_losses, test_accuracies, test_f1_scores = train_and_test(model, train_dataloader, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 1.175.. Test Loss: 0.965.. Test Accuracy: 0.692.. Test F1 Score: 0.690
Epoch: 2/30.. Train Loss: 0.903.. Test Loss: 0.912.. Test Accuracy: 0.689.. Test F1 Score: 0.681
Epoch: 3/30.. Train Loss: 0.868.. Test Loss: 0.961.. Test Accuracy: 0.662.. Test F1 Score: 0.662
Epoch: 4/30.. Train Loss: 0.852.. Test Loss: 0.921.. Test Accuracy: 0.695.. Test F1 Score: 0.696
Epoch: 5/30.. Train Loss: 0.835.. Test Loss: 0.936.. Test Accuracy: 0.648.. Test F1 Score: 0.658
Epoch: 6/30.. Train Loss: 0.833.. Test Loss: 0.935.. Test Accuracy: 0.689.. Test F1 Score: 0.685
Epoch: 7/30.. Train Loss: 0.823.. Test Loss: 0.939.. Test Accuracy: 0.639.. Test F1 Score: 0.643
Epoch: 8/30.. Train Loss: 0.821.. Test Loss: 0.942.. Test Accuracy: 0.691.. Test F1 Score: 0.681
Epoch: 9/30.. Train Loss: 0.815.. Test Loss: 0.909.. Test Accuracy: 0.655.. Test F1 Score: 0.652
Epoch: 10/30.. Train Loss: 0.816.. Test Loss: 0.922.. Test Accuracy: 0.676.. Test F1 Score: 0.677
Epoch: 11/30.. Train Loss: 0.

### 75% Train Data

In [85]:
model = TCN(num_inputs, num_channels, kernel_size, num_classes=4)

#load pretrained model
model.load_state_dict(torch.load('./models/tcn_join_20231218-2014.pt'))

model.fc = nn.Linear(in_features=model.fc.in_features, out_features=num_classes)
model.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)

# train and test model
num_epochs = 30
train_losses_75, test_losses_75, test_accuracies_75, test_f1_scores_75 = train_and_test(model, train_dataloader_75, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 1.271.. Test Loss: 1.023.. Test Accuracy: 0.638.. Test F1 Score: 0.626
Epoch: 2/30.. Train Loss: 0.963.. Test Loss: 0.950.. Test Accuracy: 0.667.. Test F1 Score: 0.655
Epoch: 3/30.. Train Loss: 0.911.. Test Loss: 0.930.. Test Accuracy: 0.663.. Test F1 Score: 0.660
Epoch: 4/30.. Train Loss: 0.891.. Test Loss: 0.916.. Test Accuracy: 0.664.. Test F1 Score: 0.657
Epoch: 5/30.. Train Loss: 0.880.. Test Loss: 0.958.. Test Accuracy: 0.674.. Test F1 Score: 0.659
Epoch: 6/30.. Train Loss: 0.866.. Test Loss: 0.935.. Test Accuracy: 0.684.. Test F1 Score: 0.671
Epoch: 7/30.. Train Loss: 0.855.. Test Loss: 0.926.. Test Accuracy: 0.687.. Test F1 Score: 0.683
Epoch: 8/30.. Train Loss: 0.856.. Test Loss: 0.939.. Test Accuracy: 0.682.. Test F1 Score: 0.673
Epoch: 9/30.. Train Loss: 0.847.. Test Loss: 0.915.. Test Accuracy: 0.699.. Test F1 Score: 0.698
Epoch: 10/30.. Train Loss: 0.840.. Test Loss: 0.934.. Test Accuracy: 0.649.. Test F1 Score: 0.635
Epoch: 11/30.. Train Loss: 0.

### 50% Train Data

In [86]:
model = TCN(num_inputs, num_channels, kernel_size, num_classes=4)

#load pretrained model
model.load_state_dict(torch.load('./models/tcn_join_20231218-2014.pt'))

model.fc = nn.Linear(in_features=model.fc.in_features, out_features=num_classes)
model.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)

# train and test model
num_epochs = 30
train_losses_50, test_losses_50, test_accuracies_50, test_f1_scores_50 = train_and_test(model, train_dataloader_50, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 1.306.. Test Loss: 1.033.. Test Accuracy: 0.668.. Test F1 Score: 0.650
Epoch: 2/30.. Train Loss: 0.921.. Test Loss: 0.998.. Test Accuracy: 0.647.. Test F1 Score: 0.644
Epoch: 3/30.. Train Loss: 0.854.. Test Loss: 0.944.. Test Accuracy: 0.663.. Test F1 Score: 0.662
Epoch: 4/30.. Train Loss: 0.822.. Test Loss: 0.985.. Test Accuracy: 0.681.. Test F1 Score: 0.683
Epoch: 5/30.. Train Loss: 0.804.. Test Loss: 0.963.. Test Accuracy: 0.666.. Test F1 Score: 0.668
Epoch: 6/30.. Train Loss: 0.790.. Test Loss: 0.954.. Test Accuracy: 0.669.. Test F1 Score: 0.671
Epoch: 7/30.. Train Loss: 0.786.. Test Loss: 0.973.. Test Accuracy: 0.662.. Test F1 Score: 0.666
Epoch: 8/30.. Train Loss: 0.772.. Test Loss: 1.008.. Test Accuracy: 0.652.. Test F1 Score: 0.660
Epoch: 9/30.. Train Loss: 0.770.. Test Loss: 0.965.. Test Accuracy: 0.654.. Test F1 Score: 0.657
Epoch: 10/30.. Train Loss: 0.759.. Test Loss: 0.948.. Test Accuracy: 0.682.. Test F1 Score: 0.682
Epoch: 11/30.. Train Loss: 0.

### 25% Train Data

In [87]:
model = TCN(num_inputs, num_channels, kernel_size, num_classes=4)

#load pretrained model
model.load_state_dict(torch.load('./models/tcn_join_20231218-2014.pt'))

model.fc = nn.Linear(in_features=model.fc.in_features, out_features=num_classes)
model.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)

# train and test model
num_epochs = 30
train_losses_25, test_losses_25, test_accuracies_25, test_f1_scores_25 = train_and_test(model, train_dataloader_25, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 1.580.. Test Loss: 1.187.. Test Accuracy: 0.611.. Test F1 Score: 0.592
Epoch: 2/30.. Train Loss: 0.981.. Test Loss: 1.078.. Test Accuracy: 0.630.. Test F1 Score: 0.625
Epoch: 3/30.. Train Loss: 0.809.. Test Loss: 1.073.. Test Accuracy: 0.642.. Test F1 Score: 0.645
Epoch: 4/30.. Train Loss: 0.718.. Test Loss: 1.087.. Test Accuracy: 0.644.. Test F1 Score: 0.647
Epoch: 5/30.. Train Loss: 0.685.. Test Loss: 1.107.. Test Accuracy: 0.630.. Test F1 Score: 0.635
Epoch: 6/30.. Train Loss: 0.652.. Test Loss: 1.081.. Test Accuracy: 0.639.. Test F1 Score: 0.644
Epoch: 7/30.. Train Loss: 0.631.. Test Loss: 1.159.. Test Accuracy: 0.613.. Test F1 Score: 0.608
Epoch: 8/30.. Train Loss: 0.608.. Test Loss: 1.114.. Test Accuracy: 0.639.. Test F1 Score: 0.646
Epoch: 9/30.. Train Loss: 0.608.. Test Loss: 1.182.. Test Accuracy: 0.631.. Test F1 Score: 0.637
Epoch: 10/30.. Train Loss: 0.598.. Test Loss: 1.153.. Test Accuracy: 0.628.. Test F1 Score: 0.635
Epoch: 11/30.. Train Loss: 0.

### 10% Train Data

In [88]:
model = TCN(num_inputs, num_channels, kernel_size, num_classes=4)

#load pretrained model
model.load_state_dict(torch.load('./models/tcn_join_20231218-2014.pt'))

model.fc = nn.Linear(in_features=model.fc.in_features, out_features=num_classes)
model.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)

# train and test model
num_epochs = 30
train_losses_10, test_losses_10, test_accuracies_10, test_f1_scores_10 = train_and_test(model, train_dataloader_10, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 1.703.. Test Loss: 1.335.. Test Accuracy: 0.559.. Test F1 Score: 0.550
Epoch: 2/30.. Train Loss: 1.040.. Test Loss: 1.219.. Test Accuracy: 0.575.. Test F1 Score: 0.581
Epoch: 3/30.. Train Loss: 0.820.. Test Loss: 1.160.. Test Accuracy: 0.550.. Test F1 Score: 0.553
Epoch: 4/30.. Train Loss: 0.705.. Test Loss: 1.188.. Test Accuracy: 0.542.. Test F1 Score: 0.546
Epoch: 5/30.. Train Loss: 0.630.. Test Loss: 1.156.. Test Accuracy: 0.560.. Test F1 Score: 0.564
Epoch: 6/30.. Train Loss: 0.580.. Test Loss: 1.207.. Test Accuracy: 0.546.. Test F1 Score: 0.556
Epoch: 7/30.. Train Loss: 0.546.. Test Loss: 1.315.. Test Accuracy: 0.529.. Test F1 Score: 0.537
Epoch: 8/30.. Train Loss: 0.525.. Test Loss: 1.180.. Test Accuracy: 0.578.. Test F1 Score: 0.582
Epoch: 9/30.. Train Loss: 0.509.. Test Loss: 1.232.. Test Accuracy: 0.560.. Test F1 Score: 0.571
Epoch: 10/30.. Train Loss: 0.479.. Test Loss: 1.223.. Test Accuracy: 0.573.. Test F1 Score: 0.580
Epoch: 11/30.. Train Loss: 0.

## Fine Tune TCN Last Temporal Block

### Full Train Data

In [89]:
model = TCN(num_inputs, num_channels, kernel_size, num_classes=4)

#load pretrained model
model.load_state_dict(torch.load('./models/tcn_join_20231218-2014.pt'))

model.fc = nn.Linear(in_features=model.fc.in_features, out_features=num_classes)

# Freezing all layers initially
for param in model.parameters():
    param.requires_grad = False

# Unfreeze layers from the last TemporalBlock's conv2 onwards
num_levels = len(model.tcn)  # Number of TemporalBlocks in your TCN
for i, block in enumerate(model.tcn):
    if i == num_levels - 1:  # Check if it's the last TemporalBlock
        # Unfreeze the conv2 layer and any subsequent layers within this block
        unfreeze = False
        for name, param in block.named_parameters():
            if 'conv2' in name:
                unfreeze = True
            if unfreeze:
                param.requires_grad = True

# Unfreeze the classification layer
for param in model.fc.parameters():
    param.requires_grad = True

model.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

# train and test model
num_epochs = 30
train_losses, test_losses, test_accuracies, test_f1_scores = train_and_test(model, train_dataloader, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 0.987.. Test Loss: 1.003.. Test Accuracy: 0.637.. Test F1 Score: 0.641
Epoch: 2/30.. Train Loss: 0.819.. Test Loss: 0.922.. Test Accuracy: 0.688.. Test F1 Score: 0.683
Epoch: 3/30.. Train Loss: 0.779.. Test Loss: 0.960.. Test Accuracy: 0.701.. Test F1 Score: 0.691
Epoch: 4/30.. Train Loss: 0.752.. Test Loss: 0.930.. Test Accuracy: 0.702.. Test F1 Score: 0.702
Epoch: 5/30.. Train Loss: 0.745.. Test Loss: 0.940.. Test Accuracy: 0.694.. Test F1 Score: 0.697
Epoch: 6/30.. Train Loss: 0.719.. Test Loss: 0.959.. Test Accuracy: 0.691.. Test F1 Score: 0.692
Epoch: 7/30.. Train Loss: 0.709.. Test Loss: 0.910.. Test Accuracy: 0.711.. Test F1 Score: 0.708
Epoch: 8/30.. Train Loss: 0.706.. Test Loss: 0.937.. Test Accuracy: 0.682.. Test F1 Score: 0.681
Epoch: 9/30.. Train Loss: 0.705.. Test Loss: 0.886.. Test Accuracy: 0.709.. Test F1 Score: 0.709
Epoch: 10/30.. Train Loss: 0.693.. Test Loss: 1.055.. Test Accuracy: 0.681.. Test F1 Score: 0.682
Epoch: 11/30.. Train Loss: 0.

### 75% Train Data

In [90]:
model = TCN(num_inputs, num_channels, kernel_size, num_classes=4)

#load pretrained model
model.load_state_dict(torch.load('./models/tcn_join_20231218-2014.pt'))

model.fc = nn.Linear(in_features=model.fc.in_features, out_features=num_classes)

# Freezing all layers initially
for param in model.parameters():
    param.requires_grad = False

# Unfreeze layers from the last TemporalBlock's conv2 onwards
num_levels = len(model.tcn)  # Number of TemporalBlocks in your TCN
for i, block in enumerate(model.tcn):
    if i == num_levels - 1:  # Check if it's the last TemporalBlock
        # Unfreeze the conv2 layer and any subsequent layers within this block
        unfreeze = False
        for name, param in block.named_parameters():
            if 'conv2' in name:
                unfreeze = True
            if unfreeze:
                param.requires_grad = True

# Unfreeze the classification layer
for param in model.fc.parameters():
    param.requires_grad = True

model.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

# train and test model
num_epochs = 30
train_losses_75, test_losses_75, test_accuracies_75, test_f1_scores_75 = train_and_test(model, train_dataloader_75, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)
    

Epoch: 1/30.. Train Loss: 1.026.. Test Loss: 0.948.. Test Accuracy: 0.684.. Test F1 Score: 0.675
Epoch: 2/30.. Train Loss: 0.853.. Test Loss: 1.156.. Test Accuracy: 0.671.. Test F1 Score: 0.669
Epoch: 3/30.. Train Loss: 0.826.. Test Loss: 0.955.. Test Accuracy: 0.678.. Test F1 Score: 0.674
Epoch: 4/30.. Train Loss: 0.805.. Test Loss: 0.962.. Test Accuracy: 0.667.. Test F1 Score: 0.658
Epoch: 5/30.. Train Loss: 0.771.. Test Loss: 1.038.. Test Accuracy: 0.660.. Test F1 Score: 0.661
Epoch: 6/30.. Train Loss: 0.755.. Test Loss: 1.000.. Test Accuracy: 0.681.. Test F1 Score: 0.678
Epoch: 7/30.. Train Loss: 0.744.. Test Loss: 1.045.. Test Accuracy: 0.681.. Test F1 Score: 0.678
Epoch: 8/30.. Train Loss: 0.727.. Test Loss: 1.055.. Test Accuracy: 0.653.. Test F1 Score: 0.654
Epoch: 9/30.. Train Loss: 0.718.. Test Loss: 0.997.. Test Accuracy: 0.675.. Test F1 Score: 0.679
Epoch: 10/30.. Train Loss: 0.709.. Test Loss: 1.029.. Test Accuracy: 0.676.. Test F1 Score: 0.675
Epoch: 11/30.. Train Loss: 0.

### 50% Train Data

In [91]:
model = TCN(num_inputs, num_channels, kernel_size, num_classes=4)

#load pretrained model
model.load_state_dict(torch.load('./models/tcn_join_20231218-2014.pt'))

model.fc = nn.Linear(in_features=model.fc.in_features, out_features=num_classes)

# Freezing all layers initially
for param in model.parameters():
    param.requires_grad = False

# Unfreeze layers from the last TemporalBlock's conv2 onwards
num_levels = len(model.tcn)  # Number of TemporalBlocks in your TCN
for i, block in enumerate(model.tcn):
    if i == num_levels - 1:  # Check if it's the last TemporalBlock
        # Unfreeze the conv2 layer and any subsequent layers within this block
        unfreeze = False
        for name, param in block.named_parameters():
            if 'conv2' in name:
                unfreeze = True
            if unfreeze:
                param.requires_grad = True

# Unfreeze the classification layer
for param in model.fc.parameters():
    param.requires_grad = True

model.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

# train and test model
num_epochs = 30
train_losses_50, test_losses_50, test_accuracies_50, test_f1_scores_50 = train_and_test(model, train_dataloader_50, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 1.006.. Test Loss: 0.986.. Test Accuracy: 0.660.. Test F1 Score: 0.660
Epoch: 2/30.. Train Loss: 0.784.. Test Loss: 0.934.. Test Accuracy: 0.662.. Test F1 Score: 0.661
Epoch: 3/30.. Train Loss: 0.741.. Test Loss: 1.049.. Test Accuracy: 0.625.. Test F1 Score: 0.623
Epoch: 4/30.. Train Loss: 0.712.. Test Loss: 1.006.. Test Accuracy: 0.671.. Test F1 Score: 0.672
Epoch: 5/30.. Train Loss: 0.692.. Test Loss: 1.035.. Test Accuracy: 0.653.. Test F1 Score: 0.655
Epoch: 6/30.. Train Loss: 0.665.. Test Loss: 0.943.. Test Accuracy: 0.672.. Test F1 Score: 0.676
Epoch: 7/30.. Train Loss: 0.664.. Test Loss: 1.014.. Test Accuracy: 0.687.. Test F1 Score: 0.690
Epoch: 8/30.. Train Loss: 0.654.. Test Loss: 0.998.. Test Accuracy: 0.673.. Test F1 Score: 0.675
Epoch: 9/30.. Train Loss: 0.641.. Test Loss: 0.989.. Test Accuracy: 0.678.. Test F1 Score: 0.677
Epoch: 10/30.. Train Loss: 0.634.. Test Loss: 1.040.. Test Accuracy: 0.669.. Test F1 Score: 0.674
Epoch: 11/30.. Train Loss: 0.

### 25% Train Data

In [92]:
model = TCN(num_inputs, num_channels, kernel_size, num_classes=4)

#load pretrained model
model.load_state_dict(torch.load('./models/tcn_join_20231218-2014.pt'))

model.fc = nn.Linear(in_features=model.fc.in_features, out_features=num_classes)

# Freezing all layers initially
for param in model.parameters():
    param.requires_grad = False

# Unfreeze layers from the last TemporalBlock's conv2 onwards
num_levels = len(model.tcn)  # Number of TemporalBlocks in your TCN
for i, block in enumerate(model.tcn):
    if i == num_levels - 1:  # Check if it's the last TemporalBlock
        # Unfreeze the conv2 layer and any subsequent layers within this block
        unfreeze = False
        for name, param in block.named_parameters():
            if 'conv2' in name:
                unfreeze = True
            if unfreeze:
                param.requires_grad = True

# Unfreeze the classification layer
for param in model.fc.parameters():
    param.requires_grad = True

model.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

# train and test model
num_epochs = 30
train_losses_25, test_losses_25, test_accuracies_25, test_f1_scores_25 = train_and_test(model, train_dataloader_25, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 1.064.. Test Loss: 1.297.. Test Accuracy: 0.590.. Test F1 Score: 0.563
Epoch: 2/30.. Train Loss: 0.640.. Test Loss: 1.181.. Test Accuracy: 0.638.. Test F1 Score: 0.639
Epoch: 3/30.. Train Loss: 0.565.. Test Loss: 1.229.. Test Accuracy: 0.644.. Test F1 Score: 0.640
Epoch: 4/30.. Train Loss: 0.520.. Test Loss: 1.310.. Test Accuracy: 0.636.. Test F1 Score: 0.639
Epoch: 5/30.. Train Loss: 0.492.. Test Loss: 1.302.. Test Accuracy: 0.644.. Test F1 Score: 0.653
Epoch: 6/30.. Train Loss: 0.502.. Test Loss: 1.394.. Test Accuracy: 0.649.. Test F1 Score: 0.645
Epoch: 7/30.. Train Loss: 0.469.. Test Loss: 1.439.. Test Accuracy: 0.616.. Test F1 Score: 0.613
Epoch: 8/30.. Train Loss: 0.477.. Test Loss: 1.526.. Test Accuracy: 0.626.. Test F1 Score: 0.635
Epoch: 9/30.. Train Loss: 0.451.. Test Loss: 1.582.. Test Accuracy: 0.650.. Test F1 Score: 0.653
Epoch: 10/30.. Train Loss: 0.429.. Test Loss: 1.500.. Test Accuracy: 0.625.. Test F1 Score: 0.621
Epoch: 11/30.. Train Loss: 0.

### 10% Train Data

In [93]:
model = TCN(num_inputs, num_channels, kernel_size, num_classes=4)

#load pretrained model
model.load_state_dict(torch.load('./models/tcn_join_20231218-2014.pt'))

model.fc = nn.Linear(in_features=model.fc.in_features, out_features=num_classes)

# Freezing all layers initially
for param in model.parameters():
    param.requires_grad = False

# Unfreeze layers from the last TemporalBlock's conv2 onwards
num_levels = len(model.tcn)  # Number of TemporalBlocks in your TCN
for i, block in enumerate(model.tcn):
    if i == num_levels - 1:  # Check if it's the last TemporalBlock
        # Unfreeze the conv2 layer and any subsequent layers within this block
        unfreeze = False
        for name, param in block.named_parameters():
            if 'conv2' in name:
                unfreeze = True
            if unfreeze:
                param.requires_grad = True

# Unfreeze the classification layer
for param in model.fc.parameters():
    param.requires_grad = True

model.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

# train and test model
num_epochs = 30
train_losses_10, test_losses_10, test_accuracies_10, test_f1_scores_10 = train_and_test(model, train_dataloader_10, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 1.202.. Test Loss: 1.295.. Test Accuracy: 0.549.. Test F1 Score: 0.542
Epoch: 2/30.. Train Loss: 0.559.. Test Loss: 1.323.. Test Accuracy: 0.570.. Test F1 Score: 0.573
Epoch: 3/30.. Train Loss: 0.470.. Test Loss: 1.600.. Test Accuracy: 0.567.. Test F1 Score: 0.564
Epoch: 4/30.. Train Loss: 0.427.. Test Loss: 1.359.. Test Accuracy: 0.573.. Test F1 Score: 0.574
Epoch: 5/30.. Train Loss: 0.401.. Test Loss: 1.594.. Test Accuracy: 0.564.. Test F1 Score: 0.562
Epoch: 6/30.. Train Loss: 0.379.. Test Loss: 1.439.. Test Accuracy: 0.578.. Test F1 Score: 0.567
Epoch: 7/30.. Train Loss: 0.369.. Test Loss: 1.633.. Test Accuracy: 0.611.. Test F1 Score: 0.612
Epoch: 8/30.. Train Loss: 0.333.. Test Loss: 1.706.. Test Accuracy: 0.577.. Test F1 Score: 0.581
Epoch: 9/30.. Train Loss: 0.318.. Test Loss: 1.990.. Test Accuracy: 0.569.. Test F1 Score: 0.571
Epoch: 10/30.. Train Loss: 0.300.. Test Loss: 2.392.. Test Accuracy: 0.522.. Test F1 Score: 0.512
Epoch: 11/30.. Train Loss: 0.

# Full Supervised

## Train and Test CNN Model

### Full Train Data

In [94]:
model = CNNFeatureExtractor(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# train and test model
num_epochs = 30
train_losses, test_losses, test_accuracies, test_f1_scores = train_and_test(model, train_dataloader, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 0.772.. Test Loss: 1.140.. Test Accuracy: 0.613.. Test F1 Score: 0.626
Epoch: 2/30.. Train Loss: 0.535.. Test Loss: 1.321.. Test Accuracy: 0.640.. Test F1 Score: 0.648
Epoch: 3/30.. Train Loss: 0.459.. Test Loss: 1.105.. Test Accuracy: 0.640.. Test F1 Score: 0.644
Epoch: 4/30.. Train Loss: 0.404.. Test Loss: 1.032.. Test Accuracy: 0.722.. Test F1 Score: 0.724
Epoch: 5/30.. Train Loss: 0.366.. Test Loss: 1.163.. Test Accuracy: 0.645.. Test F1 Score: 0.649
Epoch: 6/30.. Train Loss: 0.340.. Test Loss: 1.338.. Test Accuracy: 0.624.. Test F1 Score: 0.636
Epoch: 7/30.. Train Loss: 0.313.. Test Loss: 1.398.. Test Accuracy: 0.641.. Test F1 Score: 0.635
Epoch: 8/30.. Train Loss: 0.269.. Test Loss: 1.442.. Test Accuracy: 0.661.. Test F1 Score: 0.662
Epoch: 9/30.. Train Loss: 0.262.. Test Loss: 1.581.. Test Accuracy: 0.617.. Test F1 Score: 0.615
Epoch: 10/30.. Train Loss: 0.236.. Test Loss: 1.517.. Test Accuracy: 0.630.. Test F1 Score: 0.632
Epoch: 11/30.. Train Loss: 0.

### 75% Train Data

In [95]:
model = CNNFeatureExtractor(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# train and test model
num_epochs = 30
train_losses, test_losses, test_accuracies, test_f1_scores = train_and_test(model, train_dataloader_75, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 0.811.. Test Loss: 1.221.. Test Accuracy: 0.626.. Test F1 Score: 0.633
Epoch: 2/30.. Train Loss: 0.566.. Test Loss: 1.090.. Test Accuracy: 0.616.. Test F1 Score: 0.642
Epoch: 3/30.. Train Loss: 0.476.. Test Loss: 1.391.. Test Accuracy: 0.615.. Test F1 Score: 0.625
Epoch: 4/30.. Train Loss: 0.416.. Test Loss: 1.271.. Test Accuracy: 0.630.. Test F1 Score: 0.643
Epoch: 5/30.. Train Loss: 0.378.. Test Loss: 1.408.. Test Accuracy: 0.623.. Test F1 Score: 0.630
Epoch: 6/30.. Train Loss: 0.341.. Test Loss: 1.598.. Test Accuracy: 0.543.. Test F1 Score: 0.546
Epoch: 7/30.. Train Loss: 0.319.. Test Loss: 1.407.. Test Accuracy: 0.609.. Test F1 Score: 0.616
Epoch: 8/30.. Train Loss: 0.292.. Test Loss: 1.453.. Test Accuracy: 0.591.. Test F1 Score: 0.593
Epoch: 9/30.. Train Loss: 0.259.. Test Loss: 1.662.. Test Accuracy: 0.632.. Test F1 Score: 0.641
Epoch: 10/30.. Train Loss: 0.244.. Test Loss: 2.040.. Test Accuracy: 0.611.. Test F1 Score: 0.616
Epoch: 11/30.. Train Loss: 0.

### 50% Train Data

In [96]:
model = CNNFeatureExtractor(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# train and test model
num_epochs = 30
train_losses, test_losses, test_accuracies, test_f1_scores = train_and_test(model, train_dataloader_50, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 0.791.. Test Loss: 1.393.. Test Accuracy: 0.627.. Test F1 Score: 0.634
Epoch: 2/30.. Train Loss: 0.559.. Test Loss: 1.241.. Test Accuracy: 0.611.. Test F1 Score: 0.615
Epoch: 3/30.. Train Loss: 0.460.. Test Loss: 1.277.. Test Accuracy: 0.628.. Test F1 Score: 0.633
Epoch: 4/30.. Train Loss: 0.411.. Test Loss: 1.166.. Test Accuracy: 0.642.. Test F1 Score: 0.652
Epoch: 5/30.. Train Loss: 0.363.. Test Loss: 1.360.. Test Accuracy: 0.652.. Test F1 Score: 0.653
Epoch: 6/30.. Train Loss: 0.324.. Test Loss: 1.551.. Test Accuracy: 0.628.. Test F1 Score: 0.633
Epoch: 7/30.. Train Loss: 0.293.. Test Loss: 1.390.. Test Accuracy: 0.672.. Test F1 Score: 0.678
Epoch: 8/30.. Train Loss: 0.270.. Test Loss: 1.475.. Test Accuracy: 0.659.. Test F1 Score: 0.666
Epoch: 9/30.. Train Loss: 0.236.. Test Loss: 1.676.. Test Accuracy: 0.632.. Test F1 Score: 0.636
Epoch: 10/30.. Train Loss: 0.221.. Test Loss: 1.809.. Test Accuracy: 0.628.. Test F1 Score: 0.634
Epoch: 11/30.. Train Loss: 0.

### 25% Train Data

In [97]:
model = CNNFeatureExtractor(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# train and test model
num_epochs = 30
train_losses, test_losses, test_accuracies, test_f1_scores = train_and_test(model, train_dataloader_25, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 0.754.. Test Loss: 2.081.. Test Accuracy: 0.482.. Test F1 Score: 0.506
Epoch: 2/30.. Train Loss: 0.409.. Test Loss: 1.901.. Test Accuracy: 0.500.. Test F1 Score: 0.505
Epoch: 3/30.. Train Loss: 0.318.. Test Loss: 1.966.. Test Accuracy: 0.506.. Test F1 Score: 0.522
Epoch: 4/30.. Train Loss: 0.274.. Test Loss: 2.445.. Test Accuracy: 0.489.. Test F1 Score: 0.493
Epoch: 5/30.. Train Loss: 0.226.. Test Loss: 2.497.. Test Accuracy: 0.517.. Test F1 Score: 0.514
Epoch: 6/30.. Train Loss: 0.190.. Test Loss: 2.519.. Test Accuracy: 0.538.. Test F1 Score: 0.542
Epoch: 7/30.. Train Loss: 0.172.. Test Loss: 2.730.. Test Accuracy: 0.484.. Test F1 Score: 0.478
Epoch: 8/30.. Train Loss: 0.146.. Test Loss: 2.549.. Test Accuracy: 0.529.. Test F1 Score: 0.532
Epoch: 9/30.. Train Loss: 0.219.. Test Loss: 2.542.. Test Accuracy: 0.467.. Test F1 Score: 0.475
Epoch: 10/30.. Train Loss: 0.161.. Test Loss: 2.992.. Test Accuracy: 0.510.. Test F1 Score: 0.497
Epoch: 11/30.. Train Loss: 0.

### 10% Train Data

In [98]:
model = CNNFeatureExtractor(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# train and test model
num_epochs = 30
train_losses, test_losses, test_accuracies, test_f1_scores = train_and_test(model, train_dataloader_10, test_dataloader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/30.. Train Loss: 0.796.. Test Loss: 4.203.. Test Accuracy: 0.255.. Test F1 Score: 0.305
Epoch: 2/30.. Train Loss: 0.301.. Test Loss: 3.140.. Test Accuracy: 0.377.. Test F1 Score: 0.428
Epoch: 3/30.. Train Loss: 0.237.. Test Loss: 4.075.. Test Accuracy: 0.245.. Test F1 Score: 0.287
Epoch: 4/30.. Train Loss: 0.174.. Test Loss: 3.400.. Test Accuracy: 0.321.. Test F1 Score: 0.371
Epoch: 5/30.. Train Loss: 0.118.. Test Loss: 4.245.. Test Accuracy: 0.353.. Test F1 Score: 0.374
Epoch: 6/30.. Train Loss: 0.109.. Test Loss: 4.643.. Test Accuracy: 0.310.. Test F1 Score: 0.339
Epoch: 7/30.. Train Loss: 0.128.. Test Loss: 4.482.. Test Accuracy: 0.324.. Test F1 Score: 0.328
Epoch: 8/30.. Train Loss: 0.095.. Test Loss: 5.730.. Test Accuracy: 0.295.. Test F1 Score: 0.309
Epoch: 9/30.. Train Loss: 0.102.. Test Loss: 4.514.. Test Accuracy: 0.286.. Test F1 Score: 0.324
Epoch: 10/30.. Train Loss: 0.083.. Test Loss: 5.659.. Test Accuracy: 0.356.. Test F1 Score: 0.384
Epoch: 11/30.. Train Loss: 0.